In [2]:
# MA384 Final Project
# Kevin Cotellesso

# Let's play with some data!


import pandas as pd
df = pd.read_csv('example_data/example_Q3_2025/2025-07-01_reduced.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213459 entries, 0 to 213458
Columns: 197 entries, date to smart_255_raw
dtypes: bool(1), float64(187), int64(5), object(4)
memory usage: 319.4+ MB
